In [7]:
import pandas as pd
from transformers import AutoAdapterModel, AdapterConfig, AutoTokenizer
import torch
import numpy as np
from sklearn.metrics import f1_score

In [10]:
en_train = pd.read_csv('translated_train_all.csv')
en_test = pd.read_csv('translated_test_all.csv')

In [11]:
en_train

,text,labels,translated
0,Happy birthday Roman Abramovich. Treat yoursel...,positive,መልካም ልደት የሮማውያን የሮማውያን አብርሃም.በጥር ወር ውስጥ ጥሩ ወዳጃ...
1,Tampa Bay Buccaneers: Game-by-Game Predictions...,neutral,ታምፓ ቤተር ቡካነሮች-የጨዋታ-ጨዋታ-ጨዋታ ትንበያ ለ 2 ኛው ግማሽ ጊዜ ...
2,Interesting: Sacha Baron Cohen to play Freddi...,positive,የሚስብ: SACHA ባሮን ኮሮን በኋለኛው ባዮቲክ ውስጥ ፍሬድዲ ሜርኬሪ P...
3,DealBook: Clearwire Is Sought by Sprint for Sp...,neutral,Selegbobe: ማፅዳት ለትርፍ ይፈለጋል-ሐሙስ በሚሽከረከርበት ጊዜ ሐሙ...
4,BREAKING NEWS...........Man utd have placed Ho...,neutral,ሰበር ዜና ......... የሰው ልጅ UTD አዲስ ፈራጅ ማርክ ክሬቴንትበ...
...,...,...,...
9578,"Good night out with the boys,party bus down to...",positive,"ጥሩ ምሽት ከወንዶቹ, ነገ ከከተማው ጋር ነገ ከከተማይቱ (ኦስ ኦስ ቅዳሜ..."
9579,I wanna go crazy with Zayn till we see the sun...,neutral,ፀሐይን እስክናይ ድረስ ከጽን ጋር እብድ እፈልጋለሁ ...
9580,@Pike_JSpell are y'all going to come jam with ...,neutral,@Pike_jesplownsss ነገ ነገ ኮንትሮባንድ ጋር መምጣት የሚሄዱት?...
9581,@Nessaa456 the 6th chapter talks about malcolm...,neutral,@ Sensao456 የ 6 ኛ ምዕራፍ ስለ ማልኮም ኤክስ ነው እና እንደማስ...


In [8]:
am_train = pd.read_csv('am_train_translated.csv')
am_train, am_dev, am_test = np.split(
    am_train.sample(frac=1, random_state=42), [int(.7*len(am_train)), int(.8*len(am_train))])

am_train

,ID,tweet,label,eng_translated
1073,am_train_01074,ለትራምፕ ስንሳቀቅ ህውሀት ከመቀሌ ደሞ አልወሀድም ብላለች እና አሁን ህ...,negative,Habitheu says that when we laughed for the gru...
5468,am_train_05469,ለኔ ታላቅ ክብር ነው! እነበመባሌ የተሰማኝን ትልቅ ክብር በቃላት ልገልፀ...,positive,It is a great glory for me!I can't express it ...
5306,am_train_05307,በስህተት ወደ አካውንታቸው የገባውን አንድ ሚልዮን ብር በቅንነት የመለሱት...,positive,"In an error, the individual who first came to ..."
1321,am_train_01322,የሴቶች ደህንነት እንደ ዜጋ ቅድሚያ ይልተሰጠ... የችግኝን ያህል ግድ ያ...,negative,Women's Safety as a Citizen ... What is the pr...
4844,am_train_04845,ነገሮችን እንደአመጣጣቸው መቀበል የግድ ነው በዛው እሳቤ ኑሮ ትቂት ጊዜ...,positive,It is a necessity that we are coming as their ...
...,...,...,...,...
5700,am_train_05701,ጀግና ኢትዮጵያዊ ሁሌም ከእናንተ ጋር ነን,positive,Hero Ethiopian are always with you
5129,am_train_05130,ለመልካም ዕድልሲል በአውሮፕላን ሞተር ውስጥ ሳንቲም የወረወረው ግለሰብ ተ...,positive,The penny in a plane engine for fine lines!For...
3249,am_train_03250,ድርጊቱ መፈፀሙ ያልተገባ ቢሆንም ልዩ ልዩ ትንኮሳዎች ቢፈታተኑንም ቅሉ ዳ...,neutral,"Although the action is not unworthy, the struc..."
1208,am_train_01209,ትዕግስት ማለት ትዕግስት ማለት ብቻ አይደለም,negative,Trudy is not just Trudy


In [ ]:
combined_train = pd.concat([
    am_train[['eng_translated', 'label']].rename(columns={'eng_translated':'text', 'label':'labels'}),
    en_train[['text', 'labels']]
])

combined_test = pd.concat([
    am_test[['eng_translated', 'label']].rename(columns={'eng_translated':'text', 'label':'labels'}),
    am_dev[['eng_translated', 'label']].rename(columns={'eng_translated':'text', 'label':'labels'}),
    en_test[['text', 'labels']]
])
test_split_lengths = [('am_test', len(am_test)), ('am_dev', len(am_dev)), ('en_test', len(en_test))]

In [3]:

label2id = {"positive":0, "neutral":1, 'negative':2}
id2label = {0:"positive", 1:"neutral", 2:'negative'}

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def encode_batch(row):
    text = ' '.join(filter(lambda x:x[0]!='@', row.text.split()))
    out = tokenizer(text, max_length=100, truncation=True, padding="max_length", return_tensors='pt')
    out['labels'] = torch.LongTensor([label2id[row.labels]])[0]
    return out

train = combined_train.apply(encode_batch, axis=1).reset_index()[0]

len(train)

9583

In [5]:
test = combined_test.apply(encode_batch, axis=1).reset_index()[0]

len(test)

3138

In [6]:
model = AutoAdapterModel.from_pretrained('xlm-roberta-base')
model.add_adapter("sa")
model.train_adapter("sa")
model.add_classification_head("sa", num_labels=3)
model.set_active_adapters("sa")

NOTE: Redirects are currently not supported in Windows or MacOs.


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaAdapterModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for prediction

In [7]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True
#     # The next line is important to ensure the dataset labels are properly passed to the model
#     remove_unused_columns=False,
)

def compute_scores(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    i, output = 0, dict()
    for name, split_length in test_split_lengths:
        s = np.s_[i:i+split_length]
        split_preds = preds[s]
        split_labels = p.label_ids[s]
        output[f'{name}_acc'] = (split_preds==split_labels).mean()
        output[f'{name}_weighted_f1'] = f1_score(split_labels, split_preds, average='weighted')
        i += split_length
    return output

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_scores,
)

In [ ]:

trainer.train()

/Users/thomas/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9583
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1800


Step,Training Loss


In [ ]:
trainer.evaluate()